### Phase 1: Ising Model

$$H = \sum_{i=1}^{N} a_i X_i + \sum_{i=1}^{N} \sum_{j=1}^{i-1} J_{ij} Z_i Z_j
$$

In [1]:
import numpy as np
from qat.core import Observable, Term

def ising(N):
    np.random.seed(123)  

    terms = []

    # Generate random coefficients for the transverse field term (X)
    a_coefficients = np.random.random(N)
    for i in range(N):
        term = Term(coefficient=a_coefficients[i], pauli_op="X", qbits=[i])
        terms.append(term)

    # Generate random coefficients for the interaction term (ZZ)
    J_coefficients = np.random.random((N, N))
    for i in range(N):
        for j in range(i):
            if i != j:  # avoid duplicate terms
                term = Term(coefficient=J_coefficients[i, j], pauli_op="ZZ", qbits=[i, j])
                terms.append(term)
    ising = Observable(N, pauli_terms=terms, constant_coeff=0.0)
    return ising

In [2]:
model = ising(4) 
print("Transverse Ising Model:", model)

Transverse Ising Model: 0.6964691855978616 * (X|[0]) +
0.28613933495037946 * (X|[1]) +
0.2268514535642031 * (X|[2]) +
0.5513147690828912 * (X|[3]) +
0.48093190148436094 * (ZZ|[1, 0]) +
0.4385722446796244 * (ZZ|[2, 0]) +
0.05967789660956835 * (ZZ|[2, 1]) +
0.18249173045349998 * (ZZ|[3, 0]) +
0.17545175614749253 * (ZZ|[3, 1]) +
0.5315513738418384 * (ZZ|[3, 2])


In [3]:
print(model.nbqbits)

4


In [4]:
print(model.to_matrix)

<bound method Observable.to_matrix of Observable(terms=[Term(_coeff=TNumber(is_abstract=False, type=1, int_p=None, double_p=0.6964691855978616, string_p=None, matrix_p=None, serialized_p=None, complex_p=None), op='X', qbits=[0], _do_validity_check=True), Term(_coeff=TNumber(is_abstract=False, type=1, int_p=None, double_p=0.28613933495037946, string_p=None, matrix_p=None, serialized_p=None, complex_p=None), op='X', qbits=[1], _do_validity_check=True), Term(_coeff=TNumber(is_abstract=False, type=1, int_p=None, double_p=0.2268514535642031, string_p=None, matrix_p=None, serialized_p=None, complex_p=None), op='X', qbits=[2], _do_validity_check=True), Term(_coeff=TNumber(is_abstract=False, type=1, int_p=None, double_p=0.5513147690828912, string_p=None, matrix_p=None, serialized_p=None, complex_p=None), op='X', qbits=[3], _do_validity_check=True), Term(_coeff=TNumber(is_abstract=False, type=1, int_p=None, double_p=0.48093190148436094, string_p=None, matrix_p=None, serialized_p=None, complex_p

Based on the other result of initial state that has been choosen for the ising model creat the ansazt that apply the value k for two case where it generate the value 0010 and 0011 

In [5]:
from numpy import binary_repr
from qat.lang import Program, X
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin
import scipy.optimize

In [6]:
from qat.lang.AQASM import Program, QRoutine, RY, CNOT, RX, Z, H, RZ, I
from qat.core import Observable, Term, Circuit
from qat.lang.AQASM.gates import Gate
import matplotlib as mpl
import numpy as np
from typing import Optional, List
import warnings


In [7]:
def HEA_slow(
    nqbits: int,
    theta: List[float],
    k: int,
    n_cycles: int = 1,
    rotation_gates: List[Gate] = None,
    entangling_gate: Gate = CNOT,
    
) -> Circuit: #linear entanglement
    """
    This Hardware Efficient Ansatz has the reference from "Nonia Vaquero Sabater et al. Simulating molecules 
    with variational quantum eigensolvers. 2022" -Figure 6 -Link 
    "https://uvadoc.uva.es/bitstream/handle/10324/57885/TFM-G1748.pdf?sequence=1"

    Args:
        nqbits (int): Number of qubits of the circuit.
        n_cycles (int): Number of layers.
        rotation_gates (List[Gate]): Parametrized rotation gates to include around the entangling gate. Defaults to :math:`RY`. Must
            be of arity 1.
        entangling_gate (Gate): The 2-qubit entangler. Must be of arity 2. Defaults to :math:`CNOT`.
    """

    if rotation_gates is None:
        rotation_gates = [RZ]

    n_rotations = len(rotation_gates)

    prog = Program()
    reg = prog.qalloc(nqbits)
    theta = [prog.new_var(float, rf"\theta_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
    # the angle parameters were randomly sampled from as index variable
    #theta = [np.random.uniform(0, 2*np.pi) for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
    print("Hello")
    ind_theta = 0

    ket_value = binary_repr(k)
    padded_ket_value = ket_value.zfill(nqbits)
    list_ket_value = [int(c) for c in padded_ket_value]

        # Apply X gates based on the binary representation of the current value
    for j in range(min(nqbits, len(list_ket_value))):
        if list_ket_value[j] == 1:
            prog.apply(X, reg[j])
        if list_ket_value[j] == 0:
            prog.apply(I, reg[j])

    
    for i in range(nqbits):

        for rot in rotation_gates:

            prog.apply(rot(theta[ind_theta]), reg[i])
            ind_theta += 1
    
    for k in range(n_cycles):


        for i in range(nqbits - 1):
            prog.apply(CNOT, reg[i], reg[i+1])
            
        for i in range(nqbits):
            for rot in rotation_gates:
                            
                prog.apply(rot(theta[ind_theta]), reg[i])
                ind_theta += 1
    print(theta[ind_theta])
    return prog.to_circ()

In [15]:
n_rotations = 2
nqbits =4
n_cycles =3

theta_0 = [np.random.uniform(0, 2*np.pi) for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
theta_init = [0, 0, 0.5]
print(theta_0)
circ_Linear = HEA_slow(4, theta_0, 1, 3, [RX,RZ], CNOT)

[5.292593627669437, 0.5227295283213563, 4.79836080870194, 1.5310009843438048, 1.220338852225453, 3.5968531443031524, 0.6013794780921328, 5.562672506903891, 3.9411215251360305, 4.545359032752488, 0.10134279452259236, 3.734925651048238, 3.498384540106954, 0.9987729005246175, 0.9617704115943849, 4.370140915882929, 2.0028685266647486, 4.347777593890245, 3.4832926891731772, 2.4438485325496067, 5.8127788659370685, 5.288368558095867, 2.245595139805469, 0.2738932448605824, 1.9149142809540372, 2.501874426156342, 4.429386965658504, 6.254021789692715, 2.236279054882737, 4.791229219598553, 3.7270404867018394, 4.346090578542644, 0.9495617881049407, 2.5062136618392503, 1.5133422377246264, 2.157997781130901, 3.2240792791633512, 4.188525578996096, 0.6654426372146214, 0.822437230796549, 2.0230638157584457, 4.156731319895352, 5.318755477057054, 3.4762184199159]
Hello
\theta_{32}


In [9]:
minVQEenergy_Linear = []

linalg_qpu = get_default_qpu()
#Plot design
marker = ['1', '2', '3']


methods = ["BFGS"]

resultB = {} # for HEA_Linear

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin

# HEA_QLM
jobA = circ_Linear.to_job(job_type="OBS",
                  observable=model,
                  nbshots=0)
for method in methods:
    optimizer_scipy = ScipyMinimizePlugin(method="BFGS", x0=theta_0,
                                          tol=1e-3,
                                          options={"maxiter": 50, "disp": True},
                                          )
    qpu = optimizer_scipy | linalg_qpu
    resultB[method] = qpu.submit(jobA)
    minVQEenergy_Linear.append((method, resultB[method].value))
    print("Minimum VQE energy for QLM HEA (%s) = %s" % (method, resultB[method].value))



Optimization terminated successfully.
         Current function value: -2.141838
         Iterations: 47
         Function evaluations: 2205
         Gradient evaluations: 49
Minimum VQE energy for QLM HEA (BFGS) = -2.1418377628572385


In [14]:
theta_VQE =  resultB[method].meta_data["parameters"]
print(theta_VQE)

[3.520299841674902, 6.03544072018923, 4.107670044819194, 4.588842417470924, 4.5061880337631575, 1.9211158652726992, 3.1113385224291124, 1.6199757221560476, 1.4814350759443753, 3.712281456361877, 0.15210855634263426, 2.9804504741431406, 1.6856111901627262, 1.8581483039590228, 2.1739280809700836, 2.5248949206282876, 2.59777835041371, 6.2173932296024494, 5.5749094791653855, 3.106649623053937, 3.1188585294756335, 0.2581931552973468, 2.0529008373623703, 1.6639527180667182, 6.273184145274121, 1.5736567243921096, 3.03499318306052, 6.192453843271744, 3.264020828962286, 3.8509293736002586, 0.7579311730313538, 5.192052341290803, 3.7891386258243793, 4.168877742541969, 2.1536479143704055, 1.9108467312356843, 2.620227811065257, 4.28073832894235, 4.599482982627314, 3.31289835032298, 4.8285657745757495, 4.790793999123275, 5.992908556632375, 6.058171272140831]


In [32]:
for i in range (len(theta_VQE)):
    print(theta_VQE[i])

[
3
.
5
2
0
2
9
9
8
4
1
6
7
4
9
0
2
,
 
6
.
0
3
5
4
4
0
7
2
0
1
8
9
2
3
,
 
4
.
1
0
7
6
7
0
0
4
4
8
1
9
1
9
4
,
 
4
.
5
8
8
8
4
2
4
1
7
4
7
0
9
2
4
,
 
4
.
5
0
6
1
8
8
0
3
3
7
6
3
1
5
7
5
,
 
1
.
9
2
1
1
1
5
8
6
5
2
7
2
6
9
9
2
,
 
3
.
1
1
1
3
3
8
5
2
2
4
2
9
1
1
2
4
,
 
1
.
6
1
9
9
7
5
7
2
2
1
5
6
0
4
7
6
,
 
1
.
4
8
1
4
3
5
0
7
5
9
4
4
3
7
5
3
,
 
3
.
7
1
2
2
8
1
4
5
6
3
6
1
8
7
7
,
 
0
.
1
5
2
1
0
8
5
5
6
3
4
2
6
3
4
2
6
,
 
2
.
9
8
0
4
5
0
4
7
4
1
4
3
1
4
0
6
,
 
1
.
6
8
5
6
1
1
1
9
0
1
6
2
7
2
6
2
,
 
1
.
8
5
8
1
4
8
3
0
3
9
5
9
0
2
2
8
,
 
2
.
1
7
3
9
2
8
0
8
0
9
7
0
0
8
3
6
,
 
2
.
5
2
4
8
9
4
9
2
0
6
2
8
2
8
7
6
,
 
2
.
5
9
7
7
7
8
3
5
0
4
1
3
7
1
,
 
6
.
2
1
7
3
9
3
2
2
9
6
0
2
4
4
9
4
,
 
5
.
5
7
4
9
0
9
4
7
9
1
6
5
3
8
5
5
,
 
3
.
1
0
6
6
4
9
6
2
3
0
5
3
9
3
7
,
 
3
.
1
1
8
8
5
8
5
2
9
4
7
5
6
3
3
5
,
 
0
.
2
5
8
1
9
3
1
5
5
2
9
7
3
4
6
8
,
 
2
.
0
5
2
9
0
0
8
3
7
3
6
2
3
7
0
3
,
 
1
.
6
6
3
9
5
2
7
1
8
0
6
6
7
1
8
2
,
 
6
.
2
7
3
1
8
4
1
4
5
2
7
4
1
2
1
,
 
1
.
5
7
3
6
5
6


In [30]:
print(len(theta_VQE))

61.142857142857146


In [26]:
print(theta_0)

[5.292593627669437, 0.5227295283213563, 4.79836080870194, 1.5310009843438048, 1.220338852225453, 3.5968531443031524, 0.6013794780921328, 5.562672506903891, 3.9411215251360305, 4.545359032752488, 0.10134279452259236, 3.734925651048238, 3.498384540106954, 0.9987729005246175, 0.9617704115943849, 4.370140915882929, 2.0028685266647486, 4.347777593890245, 3.4832926891731772, 2.4438485325496067, 5.8127788659370685, 5.288368558095867, 2.245595139805469, 0.2738932448605824, 1.9149142809540372, 2.501874426156342, 4.429386965658504, 6.254021789692715, 2.236279054882737, 4.791229219598553, 3.7270404867018394, 4.346090578542644, 0.9495617881049407, 2.5062136618392503, 1.5133422377246264, 2.157997781130901, 3.2240792791633512, 4.188525578996096, 0.6654426372146214, 0.822437230796549, 2.0230638157584457, 4.156731319895352, 5.318755477057054, 3.4762184199159]


In [33]:
print(len(theta_0))

44


In [17]:
for sample in qpu.submit(jobA):
    print(f"State {sample.state} amplitude {sample.amplitude}")

Optimization terminated successfully.
         Current function value: -2.141838
         Iterations: 47
         Function evaluations: 2205
         Gradient evaluations: 49


AttributeError: 'NoneType' object has no attribute '__iter__'

In [35]:
def HEA_slow1(
    nqbits: int,
    theta: List[float],
    k: int,
    n_cycles: int = 1,
    rotation_gates: List[Gate] = None,
    entangling_gate: Gate = CNOT,
    
) -> Circuit: #linear entanglement
    """
    This Hardware Efficient Ansatz has the reference from "Nonia Vaquero Sabater et al. Simulating molecules 
    with variational quantum eigensolvers. 2022" -Figure 6 -Link 
    "https://uvadoc.uva.es/bitstream/handle/10324/57885/TFM-G1748.pdf?sequence=1"

    Args:
        nqbits (int): Number of qubits of the circuit.
        n_cycles (int): Number of layers.
        rotation_gates (List[Gate]): Parametrized rotation gates to include around the entangling gate. Defaults to :math:`RY`. Must
            be of arity 1.
        entangling_gate (Gate): The 2-qubit entangler. Must be of arity 2. Defaults to :math:`CNOT`.
    """

    if rotation_gates is None:
        rotation_gates = [RZ]

    n_rotations = len(rotation_gates)

    prog = Program()
    reg = prog.qalloc(nqbits)
    theta = [prog.new_var(float, rf"\theta_{{{i}}}") for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
    # the angle parameters were randomly sampled from as index variable
    #theta = [np.random.uniform(0, 2*np.pi) for i in range(n_rotations * (nqbits + 2 * (nqbits - 1) * n_cycles))]
    print("Hello")
    ind_theta = 0

    ket_value = binary_repr(k)
    padded_ket_value = ket_value.zfill(nqbits)
    list_ket_value = [int(c) for c in padded_ket_value]

        # Apply X gates based on the binary representation of the current value
    for j in range(min(nqbits, len(list_ket_value))):
        if list_ket_value[j] == 1:
            prog.apply(X, reg[j])
        if list_ket_value[j] == 0:
            prog.apply(I, reg[j])

    
    for i in range(nqbits):

        for rot in rotation_gates:

            prog.apply(rot(theta[ind_theta]), reg[i])
            ind_theta += 1
    
    for k in range(n_cycles):


        for i in range(nqbits - 1):
            prog.apply(CNOT, reg[i], reg[i+1])
            
        for i in range(nqbits):
            for rot in rotation_gates:
                            
                prog.apply(rot(theta[ind_theta]), reg[i])
                ind_theta += 1
    print(theta[ind_theta])
    return prog.to_circ()

In [36]:
circ2 = HEA_slow1(4, theta_VQE, 1, 3, [RX,RZ], CNOT)

Hello
\theta_{32}


In [37]:
# Build a job
job = circ2.to_job()

from qat.qpus import get_default_qpu

# Evaluate the job and print the output probabilities
result = get_default_qpu().submit(job)
for sample in result:
    print(sample.state, sample.probability)

PluginException: PluginException(code=None, modulename='qat.plugins.junction', message='Caught exception from junction: "\'<\' not supported between instances of \'NoneType\' and \'float\'" [see /tmp/tmp9606c8qm.junction.log for traceback info]', file=None, line=None)

In [23]:
from qat.qpus import get_default_qpu
mypylinalgqpu = get_default_qpu()

job = circ2.to_job()

for sample in mypylinalgqpu.submit(job):
    print("State %s Probability %s Amplitude %s" % (sample.state, sample.probability, sample.amplitude))

NameError: name 'submit' is not defined

In [ ]:
from numpy import binary_repr
from qat.lang import Program, X
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin
import scipy.optimize
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from qat.qpus import get_default_qpu
from qat.core import Batch, Job

from qat.fermion.transforms import transform_to_jw_basis
from qat.fermion.hamiltonians import make_embedded_model
from qat.fermion.circuits import make_shallow_circ, make_ldca_circ

from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation
from qat.fermion.chemistry import MolecularHamiltonian, MoleculeInfo
from qat.fermion.trotterisation import make_trotterisation_routine
from qat.fermion.chemistry.ucc import construct_ucc_ansatz, guess_init_params, get_hf_ket, get_cluster_ops
from qat.qpus import get_default_qpu

from qat.plugins import ScipyMinimizePlugin, MultipleLaunchesAnalyzer

In [ ]:
import scipy.optimize


In [ ]:
def energy_Linear_theta_s(nqbits,theta, k, n_cycles, rotational_gate, entangling_gate):
    print("Hello")
    circ_Linear_s = HEA_slow(model.nbqbits,theta, k, n_cycles, [RX,RZ], CNOT)
    print("Hello")
    jobE_s = circ_Linear_s.to_job(job_type="OBS",observable=model)
    qpu = get_default_qpu()
    resE_s = qpu.submit(jobE_s)
    energy_Linear_theta_s = resE_s.value
    print("Energy value for this iterative is", energy_Linear_theta_s)

    return energy_Linear_theta_s

In [ ]:
theta_0 = np.random.random(size=model.nbqbits * 2)
nbqbits =4
energy_Linear_theta_s(nbqbits,theta, 0, 3, [RX,RZ], CNOT),

opt_result_Linear_s = scipy.optimize.minimize(
            lambda theta: energy_Linear_theta_s(nbqbits,theta, 0, 3, [RX,RZ], CNOT),
            x0= theta_0,
            method= "SLSQP",
            tol=1e-2,
            options={"maxiter": 10, "disp": True})
print(opt_result_Linear_s)

NameError: name 'theta' is not defined